In [14]:
# Use a pipeline as a high-level helper
from transformers import pipeline
pipe = pipeline("text-classification", model="IMSyPP/hate_speech_en")

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-04 18:05:01.794516: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 18:05:01.794610: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 18:05:02.163500: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-04 18:05:02.909562: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow bi

In [41]:
result = pipe("I hate you")
result[0]["label"]

'LABEL_2'

In [16]:
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
# SKLearn libraries
import tensorflow as tf
import tensorflow_io as tfio
import warnings
warnings.filterwarnings('ignore')

In [17]:
# !curl -sSOL https://downloads.apache.org/kafka/3.3.2/kafka_2.13-3.3.2.tgz
# !tar -xzf kafka_2.13-3.3.2.tgz-

# !./kafka/bin/zookeeper-server-start.sh -daemon ./kafka/config/zookeeper.properties 
# !./kafka/bin/kafka-server-start.sh -daemon
# !echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
# !sleep 10

In [18]:
# !./kafka/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic cancer-train

In [32]:
def error_callback(exc):
      raise Exception('Error while sending data to kafka: {0}'.format(str(exc)))


def write_to_kafka(topic_name, items):
      count=0
      producer = KafkaProducer(bootstrap_servers=['127.0.0.1:29092'])
      for message, key in items:
        print(message.encode('utf-8'))
        producer.send(topic_name,
                      key=key.encode('utf-8'),
                      value=message.encode('utf-8')).add_errback(error_callback)
        count+=1
      producer.flush()
      print("Wrote {0} messages into topic: {1}".format(count, topic_name))


def decode_kafka_item(message):
#     print ("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition,
#                                           message.offset, message.key,
#                                           message.value))
    return message.value.decode('utf-8')

      


In [20]:
# import pytchat

# chat = pytchat.create(video_id="uIx8l2xlYVY")
# while chat.is_alive():
#     for c in chat.get().sync_items():
#         print(f"{c.datetime} [{c.author.name}]- {c.message}")
#         write_to_kafka('youtube', [(c.message, c.author.name)])


In [31]:

import pandas as pd
from pybloom_live import BloomFilter

df = pd.read_csv('hurtlex_EN.tsv', sep='\t')

grouped = df.groupby('category')
grouped.head()


# Create a Bloom filter with an appropriate size and false positive rate
bloom_filter = BloomFilter(capacity=df.shape[0], error_rate=0.001)

# Add hate speech terms to the Bloom filter
hate_speech_terms = df["lemma"]
for term in hate_speech_terms:
    bloom_filter.add(term)


def detect_hate_speech(text):
    tokens = text.split()
    for token in tokens:
        if token in bloom_filter:
            return True
    return False

# Test the hate speech detection function
text = "I will love you"
if detect_hate_speech(text):
    print("Hate speech detected!")
else:
    print("No hate speech detected.")
                 

Hate speech detected!


In [36]:
from kafka import KafkaConsumer

def read_from_kafka(topic_name):
    consumer = KafkaConsumer(
        topic_name,
        auto_offset_reset='earliest',
        bootstrap_servers=['127.0.0.1:29092'],
        consumer_timeout_ms=10000
    )
    # Continuously listen for messages
    while True:
        records = consumer.poll(timeout_ms=1000, max_records=500)
        print("Received {0} messages from topic: {1}".format(len(records), topic_name))
        for record in records:
            for message in records[record]:
                decoded_msg = decode_kafka_item(message)
                print(detect_hate_speech(decoded_msg))
# read_from_kafka('youtube')

In [20]:
import time  # to simulate a real time data, time loop

import numpy as np  # np mean, np random
import pandas as pd  # read csv, df manipulation
import plotly.express as px  # interactive charts
import streamlit as st  # 🎈 data web app development

st.set_page_config(
    page_title="Real-Time Data Science Dashboard",
    page_icon="✅",
    layout="wide",
)

# read csv from a github repo
dataset_url = "https://raw.githubusercontent.com/Lexie88rus/bank-marketing-analysis/master/bank.csv"

# read csv from a URL
@st.experimental_memo
def get_data() -> pd.DataFrame:
    return pd.read_csv(dataset_url)

df = get_data()

# dashboard title
st.title("Real-Time / Live Data Science Dashboard")

# top-level filters
job_filter = st.selectbox("Select the Job", pd.unique(df["job"]))

# creating a single-element container
placeholder = st.empty()

# dataframe filter
df = df[df["job"] == job_filter]

# near real-time / live feed simulation
for seconds in range(200):

    df["age_new"] = df["age"] * np.random.choice(range(1, 5))
    df["balance_new"] = df["balance"] * np.random.choice(range(1, 5))

    # creating KPIs
    avg_age = np.mean(df["age_new"])

    count_married = int(
        df[(df["marital"] == "married")]["marital"].count()
        + np.random.choice(range(1, 30))
    )

    balance = np.mean(df["balance_new"])

    with placeholder.container():

        # create three columns
        kpi1, kpi2, kpi3 = st.columns(3)

        # fill in those three columns with respective metrics or KPIs
        kpi1.metric(
            label="Age ⏳",
            value=round(avg_age),
            delta=round(avg_age) - 10,
        )
        
        kpi2.metric(
            label="Married Count 💍",
            value=int(count_married),
            delta=-10 + count_married,
        )
        
        kpi3.metric(
            label="A/C Balance ＄",
            value=f"$ {round(balance,2)} ",
            delta=-round(balance / count_married) * 100,
        )

        # create two columns for charts
        fig_col1, fig_col2 = st.columns(2)
        with fig_col1:
            st.markdown("### First Chart")
            fig = px.density_heatmap(
                data_frame=df, y="age_new", x="marital"
            )
            st.write(fig)
            
        with fig_col2:
            st.markdown("### Second Chart")
            fig2 = px.histogram(data_frame=df, x="age_new")
            st.write(fig2)

        st.markdown("### Detailed Data View")
        st.dataframe(df)
        time.sleep(1)

/tmp/ipykernel_29850/2635080564.py:29: UserWarning:

frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.

MovieWriter ffmpeg unavailable; using Pillow instead.


IndexError: list index out of range

Error in callback <function _draw_all_if_interactive at 0x7f4aafaca5f0> (for post_execute), with arguments args (),kwargs {}:


AttributeError: 'DataFrame' object has no attribute 'append'

<Figure size 640x480 with 0 Axes>